In [69]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [168]:
def ngram_probs(filename='raw_sentences.txt'):
    raw_word = pd.read_csv("raw_sentences.txt", sep='\t',header=None)
    raw_word = raw_word[0]
    word_list = []
    for i in range(len(raw_word)):
        raw_word[i] = raw_word[i].lower()
        data = raw_word[i].split(" ")
        for j in data:
            if j == "" or j==" ":
                data.remove(j)
        word_list += [data]

    bi_count = {}
    context_count = {}
    tri_count = {}
    for i in range(len(word_list)):
        for j in range(1,len(word_list[i])):
            if (word_list[i][j-1], word_list[i][j]) in bi_count.keys():
                bi_count[(word_list[i][j-1], word_list[i][j])] += 1
            else:
                bi_count[(word_list[i][j-1], word_list[i][j])] = 1
            if word_list[i][j-1] in context_count.keys():
                context_count[word_list[i][j-1]] += 1
            else:
                context_count[word_list[i][j-1]] = 1
        if word_list[i][len(word_list[i])-1] in context_count.keys():
            context_count[word_list[i][len(word_list[i])-1]] += 1
        else:
            context_count[word_list[i][len(word_list[i])-1]] = 1
        for j in range(2,len(word_list[i])):
            if (word_list[i][j-2], word_list[i][j-1], word_list[i][j]) in tri_count:
                tri_count[(word_list[i][j-2], word_list[i][j-1], word_list[i][j])] += 1
            else:
                tri_count[(word_list[i][j-2], word_list[i][j-1], word_list[i][j])] = 1
        
    bigram_probs = {}
    trigram_probs = {}
    for fir, sec in bi_count:
        bigram_probs[(fir, sec)] = bi_count[(fir, sec)]/ context_count[fir]
    for fir, sec, thir in tri_count:
        trigram_probs[(fir, sec, thir)] = tri_count[(fir, sec, thir)]/ (context_count[fir]*bigram_probs[(fir, sec)])
    return bigram_probs, trigram_probs

In [169]:
cnt2, cnt3 = ngram_probs()
print(cnt2[('we', 'are')])

0.19969401723166116


In [170]:
cnt2[("we","are")]

0.19969401723166116

In [160]:
("we","are") in cnt2.keys()

True

In [174]:
('we', 'are',"times") in cnt3.keys()

False

In [179]:
def prob3(bigram, cnt2=cnt2, cnt3=cnt3):
    prob = {}
    for (fir, sec, thir) in cnt3.keys():
        if (fir, sec)== bigram and (fir, sec) in cnt2.keys():
            prob[thir] = cnt3[(fir, sec, thir)]/cnt2[(fir, sec)]
    return prob

In [180]:
p = prob3(('we', 'are'))
print(p['family'])

0.022211400884495316


In [181]:
p

{',': 0.06259576612903227,
 '--': 0.002019218262226847,
 '.': 0.39374756113423515,
 '?': 0.0343267104578564,
 'a': 0.1797104253381894,
 'about': 0.058557329604578566,
 'after': 0.02019218262226847,
 'against': 0.02019218262226847,
 'all': 0.24230619146722163,
 'among': 0.002019218262226847,
 'an': 0.004038436524453694,
 'and': 0.016153746097814777,
 'another': 0.002019218262226847,
 'around': 0.002019218262226847,
 'as': 0.018172964360041625,
 'at': 0.10096091311134235,
 'back': 0.06057654786680541,
 'because': 0.002019218262226847,
 'before': 0.002019218262226847,
 'being': 0.008076873048907389,
 'best': 0.004038436524453694,
 'big': 0.004038436524453694,
 'both': 0.024230619146722164,
 'business': 0.006057654786680541,
 'but': 0.002019218262226847,
 'called': 0.004038436524453694,
 'city': 0.004038436524453694,
 'country': 0.002019218262226847,
 'day': 0.002019218262226847,
 'down': 0.028269055671175856,
 'each': 0.002019218262226847,
 'even': 0.018172964360041625,
 'family': 0.02221

In [185]:
def predict_max(starting, cnt2=cnt2, cnt3=cnt3):
    list_of_words = list(starting)
    while list_of_words[-1] != "." and len(list_of_words) < 15:
        prob = prob3((list_of_words[-2],list_of_words[-1]), cnt2, cnt3)
        Maxword = ""
        Maxprob = 0 
        for word in prob.keys():
            if Maxword == "":
                Maxprob = prob[word]
                Maxword = word
                #print(word, prob[word], Maxprob)
            if prob[word] > Maxprob:
                #print(word, prob[word], Maxprob)
                Maxprob = prob[word]
                Maxword = word
        list_of_words += [Maxword]
    
    return list_of_words

In [186]:
sent = predict_max(('we', 'are'))
assert sent[-1] == '.' or len(sent) <= 15
print(' '.join(sent))

we are going to do it .


In [ ]:
def BFS(setence, word, sent_length=10, cnt2, cnt3, prob):
    if len(setence) ==10 and setence[-1] == ".": return setence, prob
    

In [ ]:
def predict_beam(bigram, beam_size=4, sent_length=10, cnt2=cnt2, cnt3=cnt3):
    prob = prob3(bigram, cnt2, cnt3)
    check_word = ["" for i in range(beam_size)]
    check_prob = [0.0 for i in range(beam_size)]
    for word in prob.keys():
    for word in prob.keys():
        setence, prob = BFS(list(starting), word, sent_length=10, cnt2=cnt2, cnt3=cnt3)
        
    return list_of_sentence

In [ ]:
for sent in predict_beam(('we', 'are')):
assert sent[-1] == '.' or len(sent) < 10
print(' '.join(sent))